In [38]:
import random

In [2]:
random

<module 'random' from '/usr/lib/python3.5/random.py'>

In [232]:
random.seed(1)
first_rand32=random.getrandbits(32)
second_rand32=random.getrandbits(32)
print(bin(first_rand32))
print(bin(second_rand32))

random.seed(1)
first_rand64=random.getrandbits(64)
print(bin(first_rand64))
print(bin(first_rand64 & 0xFFFFFFFF)) # last 32 bits of first_rand64
print(bin(first_rand64 >>32)) # first 32 bits of first_rand64

print(first_rand32 == (first_rand64 & 0xFFFFFFFF))
print(second_rand32 == (first_rand64 >>32))

#### AGAIN
random.seed(1)
random.getrandbits(32)
random.getrandbits(32)

first_rand32=random.getrandbits(32)
second_rand32=random.getrandbits(32)
print(bin(first_rand32))
print(bin(second_rand32))

random.seed(1)
random.getrandbits(32)
random.getrandbits(32)

first_rand64=random.getrandbits(64)
print(bin(first_rand64))
print(bin(first_rand64 & 0xFFFFFFFF)) # last 32 bits of first_rand64
print(bin(first_rand64 >>32)) # first 32 bits of first_rand64

print(first_rand32 == (first_rand64 & 0xFFFFFFFF))
print(second_rand32 == (first_rand64 >>32))

0b100010011001011011000111110101
0b10010001101101110101100001001010
0b1001000110110111010110000100101000100010011001011011000111110101
0b100010011001011011000111110101
0b10010001101101110101100001001010
True
True
0b11011000111100010110101011011111
0b11001101011000010011111000110000
0b1100110101100001001111100011000011011000111100010110101011011111
0b11011000111100010110101011011111
0b11001101011000010011111000110000
True
True


## Why it's probably not solvable for randbelow(9E10)

In [13]:
random.seed(4710924109828301)

n = int(9E10)
#n = int(2**37-1) # but it's solvable for randbelow(2**37-1)

print("n=",n)
k = n.bit_length()
print("k=",k)
for i in range(624*3):
    r = random.getrandbits(k)
    print("r=",r)
    while r >= n:
        # Everytime we get here, we lose two 32bit numbers from the 624 states, which makes it harder to get a complete picture
        print("r>n, we need to getrandbits again")
        r = random.getrandbits(k)
        print("r=",r)


n= 90000000000
k= 37
r= 80832165503
r= 90705583653
r>n, we need to getrandbits again
r= 78458192218
r= 62614214559
r= 47080623768
r= 5403394540
r= 106863987153
r>n, we need to getrandbits again
r= 110153871278
r>n, we need to getrandbits again
r= 59088104708
r= 94132818641
r>n, we need to getrandbits again
r= 45463424063
r= 1055103783
r= 50901462144
r= 40013338952
r= 16335958727
r= 110626895973
r>n, we need to getrandbits again
r= 9029477694
r= 70327413599
r= 102197471889
r>n, we need to getrandbits again
r= 115721945321
r>n, we need to getrandbits again
r= 76628804563
r= 59712318286
r= 82736209024
r= 67735181548
r= 128483940438
r>n, we need to getrandbits again
r= 90241104543
r>n, we need to getrandbits again
r= 100164256515
r>n, we need to getrandbits again
r= 82738078005
r= 130248303699
r>n, we need to getrandbits again
r= 111384815567
r>n, we need to getrandbits again
r= 46879496118
r= 7855509692
r= 134414430597
r>n, we need to getrandbits again
r= 45820955280
r= 1920050927
r= 9805

In [26]:
#!/usr/bin/env python3
#
# mt19937.py - a quick and dirty implementation of the MT19937 PRNG in Python
#
#    Copyright (C) 2020  Tom Liston - email: tom.liston@bad-wolf-sec.com
#                                   - twitter: @tliston
#
#    This program is free software: you can redistribute it and/or modify
#    it under the terms of the GNU General Public License as published by
#    the Free Software Foundation, either version 3 of the License, or
#    (at your option) any later version.
#
#    This program is distributed in the hope that it will be useful,
#    but WITHOUT ANY WARRANTY; without even the implied warranty of
#    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
#    GNU General Public License for more details.
#
#    You should have received a copy of the GNU General Public License
#    along with this program.  If not, see [http://www.gnu.org/licenses/].

import random

# this is simply a python implementation of a standard Mersenne Twister PRNG.
# the parameters used, implement the MT19937 variant of the PRNG, based on the
# Mersenne prime 2^19937−1
# see https://en.wikipedia.org/wiki/Mersenne_Twister for a very good explanation
# of the math behind this...

class mt19937():
    u, d = 11, 0xFFFFFFFF
    s, b = 7, 0x9D2C5680
    t, c = 15, 0xEFC60000
    l = 18
    n = 624 # number of states
    m = 397 # one of the state indexes used when twisting states
    a = 0x9908B0DF # magic number used when twisting states
    w = 32
    r = 31
    f = 1812433253
    lower_mask = (1 << r) - 1
    upper_mask = (~lower_mask) & 0xFFFFFFFF

    
    
    def my_int32(self, x):
        return(x & 0xFFFFFFFF)

    def __init__(self, seed):

        self.MT = [0] * self.n
        self.index = self.n + 1
        self.MT[0] = self.my_int32(seed)
        for i in range(1, self.n):
            self.MT[i] = self.my_int32((self.f * (self.MT[i - 1] ^ (self.MT[i - 1] >> (self.w - 2))) + i))

    def extract_number(self):
        if self.index >= self.n:
            self.twistStates()
            self.index = 0
        y = self.MT[self.index]
        y = self.temper(y)
        self.index += 1
        return self.my_int32(y)
    
    def temper(self, y):
        # this implements the so-called "tempering matrix"
        # this, functionally, should alter the output to
        # provide a better, higher-dimensional distribution
        # of the most significant bits in the numbers extracted
        y = y ^ ((y >> self.u) & self.d)
        y = y ^ ((y << self.s) & self.b)
        y = y ^ ((y << self.t) & self.c)
        y = y ^ (y >> self.l)
        return y

    def twistStates(self): # This changes the current 624 states to their next values
        for i in range(0, self.n):
            x = (self.MT[i] & self.upper_mask) + (self.MT[(i + 1) % self.n] & self.lower_mask)
            xA = x >> 1
            if(x % 2) != 0:
                xA = xA ^ self.a
            self.MT[i] = self.MT[(i + self.m) % self.n] ^ xA
    

# so... guess what! while it isn't necessarily obvious, the
# functioning of the tempering matrix are mathematically
# reversible. this function impliments that...
#
# by using this, we can take the output of the MT PRNG, and turn
# it back into the actual values held within the MT[] array itself
# and therefore, we can "clone" the state of the PRNG from "n"
# generated random numbers...
#
# initially, figuring out the math to do this made my brain hurt.
# simplifying it caused even more pain.
# please don't ask me to explain it...
def untemper(y):
    y ^= y >> mt19937.l
    y ^= y << mt19937.t & mt19937.c
    for i in range(7):
        y ^= y << mt19937.s & mt19937.b
    for i in range(3):
        y ^= y >> mt19937.u & mt19937.d
    return y

In [64]:
###### just testing the original program 
# create our own version of an MT19937 PRNG.
myprng = mt19937(0)
# fire up Python's built-in PRNG and seed it with the time...
print("Seeding Python's built-in PRNG with the time...")
random.seed()
# generate some random numbers so we can create a random number of random numbers using Python's built-in PRNG
# so random...
count1 = random.randint(2000, 10000)
count2 = random.randint(2000, 10000)
print("Generating a random number (%i) of random numbers using Python's built-in PRNG..." % (count1))
print("We do this just to show that this method doesn't depend on being at a particular starting point.")
for i in range(count1):
    f = random.randrange(0xFFFFFFFF)
# clone that sucker...
print("Generating %i random numbers.\nWe'll use those values to create a clone of the current state of Python's built-in PRNG..." % (mt19937.n))
for i in range(mt19937.n):
    myprng.MT[i] = untemper(random.randrange(0xFFFFFFFF))
# check to make sure our cloning worked...
print("Generating a random number (%i) of additional random numbers using Python's built-in PRNG..." % (count2))
print("Generating those %i random numbers with our clone as well..." % (count2))
# generate numbers and throw 'em away...
for i in range(count2):
    f = random.randrange(0xFFFFFFFF)
    f2 = myprng.extract_number()
print("Now, we'll test the clone...")
print("\nPython       Our clone")
for i in range(200):
    r1 = random.randrange(0xFFFFFFFF)
    r2 = myprng.extract_number()
    print("%10.10i - %10.10i (%r)" % (r1, r2, (r1 == r2)))
#    assert(r1 == r2)

Seeding Python's built-in PRNG with the time...
Generating a random number (5781) of random numbers using Python's built-in PRNG...
We do this just to show that this method doesn't depend on being at a particular starting point.
Generating 624 random numbers.
We'll use those values to create a clone of the current state of Python's built-in PRNG...
Generating a random number (4520) of additional random numbers using Python's built-in PRNG...
Generating those 4520 random numbers with our clone as well...
Now, we'll test the clone...

Python       Our clone


In [27]:
# create our own version of an MT19937 PRNG.
myPRNG_full = mt19937(0)
# Seeding Python's built-in PRNG with our seed
random.seed(4710924109828301)
#random.randrange(10E9,10E10)
print(mt19937.n)
for i in range(mt19937.n):
    myPRNG_full.MT[i] = untemper(random.randrange(0xFFFFFFFF))
# now we have all 624 states, let's save them and get the next 624 states as well
# states[0] will be an array of the first 624 states, states[1] will be an array of the next 624 states etc.
states = [[0] * mt19937.n, [0] * mt19937.n, [0] * mt19937.n]  # initialize 3 "empty" arrays

for i in range(mt19937.n):
    states[0][i] = myPRNG_full.MT[i]
#     print(myPRNG_full.MT[i]) #debug
print("Twisting states")
myPRNG_full.twistStates()
for i in range(mt19937.n):
    states[1][i] = myPRNG_full.MT[i]
#     print(myPRNG_full.MT[i]) #debug
print("Twisting states")
myPRNG_full.twistStates()
for i in range(mt19937.n):
    states[2][i] = myPRNG_full.MT[i]
#     print(myPRNG_full.MT[i]) #debug



624
Twisting states
Twisting states


In [21]:
print(states[0][0:10])
print(states[1][0:10])
print(states[2][0:10])
nextrand=random.randrange(0xFFFFFFFF)
print("Next random number is", nextrand)
print("Untempered (aka equal to the MT state):", untemper(nextrand))
#print(myPRNG_full.temper(1151985231))

[2901221154, 2131571682, 3197017320, 421866341, 1434041715, 3475386739, 2504498922, 303478731, 1452909090, 4048553118]
[1151985231, 2653792599, 4132213236, 1112079699, 598655621, 2619809924, 1153997192, 176311172, 4188111642, 1487211524]
[751333919, 1513687186, 2771579250, 469482740, 1068658698, 183607433, 1768402469, 1514314239, 3393403297, 1867974574]
Next random number is 2371493812
Untempered (aka equal to the MT state): 2653792599


In [ ]:
# algorithm for twisting states
for i in range(0, self.n):
    x = (self.MT[i] & self.upper_mask) + (self.MT[(i + 1) % self.n] & self.lower_mask)
    xA = x >> 1
    if(x % 2) != 0:
        xA = xA ^ self.a
    self.MT[i] = self.MT[(i + self.m) % self.n] ^ xA

In [28]:
# let's make it a separate function
def getNextState(i):
    # get the first bit of the current number,  and the last 31 bits of the next number
    x = (states[0][i] & mt19937.upper_mask) + (states[0][(i + 1) % mt19937.n] & mt19937.lower_mask)
    # bitshift it by 1 to the right
    xA = x >> 1
    # if x is odd, xor with magic number
    if(x % 2) != 0:
        xA = xA ^ mt19937.a
    # xor it with the 397th next number
    return states[0][(i + mt19937.m) % mt19937.n] ^ xA
#print(getNextState(0))

##### This function calculates the next state based on the previous states and the twisting function
##### Basically state[1][i] = function (state[0][i],state[0][(i+1) %624],state[0][(i+397) %624])
def getNextState_static(state_i,state_i_plus_1,state_i_plus_397):
    # get the first bit of the current number,  and the last 31 bits of the next number
    x = (state_i & mt19937.upper_mask) + (state_i_plus_1 & mt19937.lower_mask)
    # bitshift it by 1 to the right
    xA = x >> 1
    # if x is odd, xor with magic number
    if(x % 2) != 0:
        xA = xA ^ mt19937.a
    # xor it with the 397th next number
    return state_i_plus_397 ^ xA
#print(getNextState_static(state_i=states[0][0],state_i_plus_1=states[0][1],state_i_plus_397=states[0][397]))

In [29]:
##### This function calculates the next state based on the previous states and the twisting function
##### Basically state[1][i] = function (state[0][i],state[0][(i+1) %624],state[0][(i+397) %624])
##### Only the 1st bit of state[0][i] is used, so this function is used to bruteforce that bit
def getNextState_unknownFirstBit(states, j, i, firstbit):
    # if i+1>623, we've reached the end of the states, make sure we use the next states
    if (i + 1) > 623:
        # we don't know the first bit of the current number (so we bruteforce it), but we know the last 31 bits of the next number
        x = (firstbit * mt19937.upper_mask) + (states[j+1][(i + 1) % mt19937.n] & mt19937.lower_mask)
    else:
        x = (firstbit * mt19937.upper_mask) + (states[j][(i + 1) % mt19937.n] & mt19937.lower_mask)

    # bitshift it by 1 to the right
    xA = x >> 1
    # if x is odd, XOR with magic number
    if(x % 2) != 0:
        xA = xA ^ mt19937.a
    # XOR it with the 397th next number
    # if i + 397 > 623, we've reached the end of the states, make sure we use the next states (which at this point should already be calculated)
    if (i + mt19937.m) > 623:
        return states[j+1][(i + mt19937.m) % mt19937.n] ^ xA
    else:
        return states[j][(i + mt19937.m) % mt19937.n] ^ xA

# possibleNextState=(getNextState_unknownFirstBit(states=states,j=0,i=623,firstbit=0), getNextState_unknownFirstBit(states=states,j=0,i=623,firstbit=1))
# print(possibleNextState)
# print(states[1][623])

# POC OR GTFO

In [34]:
# Let's do the POC
random.seed(4710924109828301) # Any seed should work, this was the one used in the challenge

# By looking at the Python source code of the random module at e.g. /usr/lib/python3.7/random.py we can see
# random.randrange(10E9,10E10) will call random._randbelow(10E10 - 10E9), or random._randbelow(9E10)
# which calls the built-in (aka C Python) method  _random.Random().getrandbits(int(9E10).bit_length()), or getrandbits(37)
# which calls the CPython function genrand_int32() two times. See e.g. https://github.com/python/cpython/blob/3.7/Modules/_randommodule.c
# This is basically equivalent to calling random.randrange(0xFFFFFFFF) twice, and keeping the entire 32 bits of the first number and the MSB 5 bits of the second number.
# The 2 numbers are combined in a 37 bit number that looks like: MSB_5bitsOfSecondRandomNumber concatenated to 32bitsOfFirstRandomNumber
# If this 37 bit number is less than 9E10 (i.e. because of random._randbelow(9E10)), what is returned is is 10E9 + (MSB_5bitsOfSecondRandomNumber concatenated to 32bitsOfFirstRandomNumber)
# The problem with solving the challenge appears if this 37 bit number is larger than or equal to 9E10, because the number is discarded and the procedure described above is repeated (i.e. genrand_int32() called again two times etc.)
# This makes building the 624 Mersenne Twister states very difficult.
# For this reason, I changed the challenge from using random.randrange(10E9,10E10) to using random.randrange(10E9,10E9 + 2**37 - 1).
# This guarantees random._randbelow(2**37-1) is called instead, where 2**37-1 is the largest 37 bit number


# Now we would interact with the server to extract enough random numbers to partially build 3x 624 states
# Since one random.randrange(10E9,10E9 + 2**37 - 1) basically calls random.getrandbits(32) twice, we need to call it 312 times for it to go through all 624 states

# First we initialize 3 "empty" arrays. partialStates[0] will be an array of the first 624 MT states, partialStates[1] will be an array of the next 624 states etc.
partialStates = [[-1] * mt19937.n, [-1] * mt19937.n, [-1] * mt19937.n]  # we initialize the states with invalid value -1

for j in range(3):
    for i in range(0,mt19937.n,2):
        nextRandomReceivedFromServer=random.randrange(10E9,10E9 + 2**37 - 1) # These would be the random numbers received from the server
        tempered = int(nextRandomReceivedFromServer - 10E9) & 0xFFFFFFFF # we want only the last 32 bits of the first random number, which is equivalent to running random.getrandbits(32) once. We basically discard the MSB 5 bits of the second call to random.getrandbits(32) in order to make this approach more general 
        partialStates[j][i] = untemper(tempered) # we will thus obtain all the even states e.g. state[0],state[2],state[4],...,state[622]
        

In [31]:
#Validating that everything's ok
print("Our MT model has these sample states:")
print(states[0][:20])
print(states[1][:20])
print(states[2][:20])


print("\nThese states were generated by the server:")
print(partialStates[0][0:20])
print(partialStates[1][0:20])
print(partialStates[2][0:20])
for i in range(0,mt19937.n,2):
    for j in range(3):
        if states[j][i] != partialStates[j][i]:
            print("Found a state that was different than our model",i,j)

Our MT model has these sample states:
[2901221154, 2131571682, 3197017320, 421866341, 1434041715, 3475386739, 2504498922, 303478731, 1452909090, 4048553118, 4071297034, 93562122, 4050341597, 2619555334, 4076001822, 1600237763, 3128542407, 4072097521, 2329797469, 2702138653]
[1151985231, 2653792599, 4132213236, 1112079699, 598655621, 2619809924, 1153997192, 176311172, 4188111642, 1487211524, 1347689326, 589134402, 695693926, 2391278992, 3059008743, 4170586443, 66415177, 3868091044, 4147508352, 3923000326]
[751333919, 1513687186, 2771579250, 469482740, 1068658698, 183607433, 1768402469, 1514314239, 3393403297, 1867974574, 3796694229, 2886227423, 1822581117, 2727994402, 4095779913, 2638341832, 1668735091, 4250545630, 612084912, 3172465192]

These states were generated by the server:
[2901221154, -1, 3197017320, -1, 1434041715, -1, 2504498922, -1, 1452909090, -1, 4071297034, -1, 4050341597, -1, 4076001822, -1, 3128542407, -1, 2329797469, -1]
[1151985231, -1, 4132213236, -1, 598655621, -1, 

In [104]:
# let's finish filling the missing states
# partialStates[1][0] is a function of 1st bit of partialStates[0][0], last 31 bits of partialStates[0][1] and all of partialStates[0][0+397]
# since we have all even states, we have partialStates[1][0], partialStates[0][0], but miss partialStates[0][1] and partialStates[0][397]. we are missing 31 + 32 = 63 bits

# partialStates[1][1] is a function of 1st bit of partialStates[0][1], last 31 bits of partialStates[0][2] and all of partialStates[0][1+397]
# since we have all even states, we have partialStates[0][2] and partialStates[0][398] but miss partialStates[0][1]. we are missing 1 bit
# as we're only missing 1 bit, we can "bruteforce" it 

partialStates[1][1]=(getNextState_unknownFirstBit(states=partialStates,j=0,i=1,firstbit=0), getNextState_unknownFirstBit(states=partialStates,j=0,i=1,firstbit=1))
print(partialStates[1][1]) # so it's either of these 2 values
print(states[1][1]) # debugging

# how can we identify which of the 2 is correct? we'll use partialStates[2][0], which we know
# partialStates[2][0] is a function of 1st bit of partialStates[1][0], last 31 bits of partialStates[1][1] and all of partialStates[1][0+397]
# since we have all even states, we have partialStates[2][0], partialStates[1][0]. We also know partialStates[1][1] is one of 2 values (calculated above). Let's calculate partialStates[1][397]

# partialStates[1][397] is a function of 1st bit of partialStates[0][397], last 31 bits of partialStates[0][398] and all of partialStates[1][(397+397) % 624]. 
# we have partialStates[0][398] and we have partialStates[1][170], we can "bruteforce" the missing bit of partialStates[0][397]
partialStates[1][397]=(getNextState_unknownFirstBit(states=partialStates,j=0,i=397,firstbit=0), getNextState_unknownFirstBit(states=partialStates,j=0,i=397,firstbit=1))
print(partialStates[1][397])
print(states[1][397]) # debugging

# now we have 2 possible values for partialStates[1][1] and 2 possible values for partialStates[1][397]. let's find which are the correct ones
# there is a 1 in 2**62 chance this is incorrect, but we'll take our chances :D
for state_i_plus_1 in partialStates[1][1]:
    for state_i_plus_397 in partialStates[1][397]:
        if partialStates[2][0]==getNextState_manual(state_i=partialStates[1][0],state_i_plus_1=state_i_plus_1,state_i_plus_397=state_i_plus_397):
            print("partialStates[1][1] = {}, partialStates[1][397] = {}".format(state_i_plus_1,state_i_plus_397))
            partialStates[1][1]=state_i_plus_1
            partialStates[1][397]=state_i_plus_397
            found=True
    if found:
        break

(2653792599, 3727534423)
2653792599
(3134592619, 4208334443)
3134592619
partialStates[1][1] = 2653792599, partialStates[1][397] = 3134592619


In [35]:
j=1
for i in range(0,mt19937.n,2):
    if partialStates[1][i+1]!= -1: # if we've already calculated it e.g. reached partialStates[1][397]
        continue
        
    partialStates[1][i+1]=(getNextState_unknownFirstBit(states=partialStates,j=0,i=i+1,firstbit=0), getNextState_unknownFirstBit(states=partialStates,j=0,i=i+1,firstbit=1))
#     print(partialStates[1][i]) # debugging
    if (i + 397 > 623 and j==1): 
        j+=1
    partialStates[j][(i+397) % 624]=(getNextState_unknownFirstBit(states=partialStates,j=j-1,i=(i+397) % 624,firstbit=0), getNextState_unknownFirstBit(states=partialStates,j=j-1,i=(i+397) % 624,firstbit=1))
#     print(partialStates[1][i+397]) # debugging

    found = False
    for state_i_plus_1 in partialStates[1][i+1]:
        for state_i_plus_397 in partialStates[j][(i+397) % 624]:
            if partialStates[2][i]==getNextState_manual(state_i=partialStates[1][i],state_i_plus_1=state_i_plus_1,state_i_plus_397=state_i_plus_397):
                print("partialStates[1][{}] = {}, partialStates[{}][{}] = {}".format(i+1,state_i_plus_1,j,(i+397) % 624,state_i_plus_397)) # debugging
                partialStates[1][i+1]=state_i_plus_1
                partialStates[j][(i+397) % 624]=state_i_plus_397
                found=True
        if found:
            break
    if not found: # we shouldn't reach this
        print("Error in the algorithm, or, less likely, the 1 in 2**62 chance was hit")

partialStates[1][1] = 2653792599, partialStates[1][397] = 3134592619
partialStates[1][3] = 1112079699, partialStates[1][399] = 1562263300
partialStates[1][5] = 2619809924, partialStates[1][401] = 832692296
partialStates[1][7] = 176311172, partialStates[1][403] = 1814465511
partialStates[1][9] = 1487211524, partialStates[1][405] = 2786181027
partialStates[1][11] = 589134402, partialStates[1][407] = 4089596404
partialStates[1][13] = 2391278992, partialStates[1][409] = 1810260917
partialStates[1][15] = 4170586443, partialStates[1][411] = 291706931
partialStates[1][17] = 3868091044, partialStates[1][413] = 1345442081
partialStates[1][19] = 3923000326, partialStates[1][415] = 1351725747
partialStates[1][21] = 1583070130, partialStates[1][417] = 3178317475
partialStates[1][23] = 3990028939, partialStates[1][419] = 1917738496
partialStates[1][25] = 2485773068, partialStates[1][421] = 3031335943
partialStates[1][27] = 1814071842, partialStates[1][423] = 779286503
partialStates[1][29] = 2722781

In [36]:
############################################
# Now we have the entire 624 states in partialStates[1]
# We can initialize our own version of an MT19937 PRNG.
############################################
solvedPRNG = mt19937(0)

# Create our MT19937 clone
print("Initialize our MT19937 PRNG clone based on the partialStates[1] we got from the random server and further calculated by us")
for i in range(mt19937.n):
    solvedPRNG.MT[i] = partialStates[1][i]
# The random server is already using states[3] so we need to move our states there as well
print("Twisting states")
solvedPRNG.twistStates()
solvedPRNG.twistStates()
solvedPRNG.index=0

# check to make sure our cloning worked...
count=1000 # let's guess the next 1000 random numbers
print("Now, we'll test the clone...")
print("\nPython       Our clone")
for i in range(count):
    nextRandomFromServer=random.randrange(10E9,10E9 + 2**37 - 1) # These would be the random numbers generated by the server
    ourRandomPart1 = solvedPRNG.extract_number() # we need all 32 bits of this one
    ourRandomPart2 = solvedPRNG.extract_number() # we only need the MSB 5 bits of this one
    ourRandomNumber = (ourRandomPart2 >> 32-5 << 32) + ourRandomPart1
    ourRandomGuess = int(ourRandomNumber + 10E9)
    print("%12.12i - %12.12i (%r)" % (nextRandomFromServer, ourRandomGuess, (nextRandomFromServer == ourRandomGuess)))

Initialize our MT19937 PRNG clone based on the partialStates[1] we got from the random server and further calculated
Twisting states
Now, we'll test the clone...

Python       Our clone
080196979509 - 080196979509 (True)
084207899254 - 084207899254 (True)
044890871295 - 044890871295 (True)
107801271675 - 107801271675 (True)
056281174346 - 056281174346 (True)
031982490588 - 031982490588 (True)
089827608393 - 089827608393 (True)
094954699751 - 094954699751 (True)
105271519335 - 105271519335 (True)
041205341666 - 041205341666 (True)
091897446733 - 091897446733 (True)
065562688034 - 065562688034 (True)
123476368165 - 123476368165 (True)
030121874257 - 030121874257 (True)
061034973263 - 061034973263 (True)
097889051196 - 097889051196 (True)
128999836801 - 128999836801 (True)
098962786699 - 098962786699 (True)
014475441346 - 014475441346 (True)
059286031759 - 059286031759 (True)
071984144432 - 071984144432 (True)
070711578387 - 070711578387 (True)
046917384364 - 046917384364 (True)
115220779